# Chapter 10: Model Context Protocol

LLM이 효과적인 에이전트로 기능하기 위해서는, 현재 데이터를 조회하고,    
외부 소프트웨어를 활용하는 등 **외부 환경과의 상호작용**이 필수적이다.

**Model Context Protocol(MCP)** 은 이러한 요구를 충족하기 위해 설계된,  
LLM이 외부 리소스와 연결되도록 하는 **표준화된 인터페이스**를 제공한다.  
이 프로토콜은 LLM과 외부 시스템 간의 통합이 **일관되고 예측 가능하게** 이루어지도록 하는  
핵심 메커니즘 역할을 한다.

## MCP Pattern Overview

**MCP(Model Context Protocol)** 패턴은

> 어떤 LLM이든, 어떤 외부 시스템·데이터베이스·툴이든  
> 매번 커스텀 연동 없이 바로 꽂아서 연결할 수 있게 해주는  
> **범용 어댑터(universal adapter)**

Gemini, OpenAI GPT, Mixtral, Claude 같은 LLM들이  
외부 애플리케이션, 데이터 소스, 툴과 **표준화된 방식으로 소통**할 수 있게 해 준다.


MCP는:

- LLM이 **컨텍스트를 얻고(context)**
- **액션을 실행하며(actions)**
- 여러 시스템과 상호작용하는 과정을

단순하고 일관된 방식으로 만들어 주는  
**범용 연결 메커니즘(universal connection mechanism)** 이라고 할 수 있다.

### MCP의 기본 구조: 클라이언트–서버 아키텍처

MCP는 **클라이언트–서버 아키텍처** 위에서 동작한다.

- **MCP 서버(MCP server)** 는 다음과 같은 요소들을 외부로 노출한다.
  - **Resources (리소스)**: 데이터  
  - **Prompts / Interactive Templates (인터랙티브 템플릿)**: 프롬프트 역할을 하는 상호작용 형식  
  - **Tools (툴)**: 실제로 실행 가능한 함수·액션

- **MCP 클라이언트(MCP client)** 는 이를 소비하는 쪽으로,
  - LLM을 호스팅하는 애플리케이션일 수도 있고  
  - 하나의 **AI 에이전트 자체**일 수도 있다.

이 표준화된 방식으로 LLM을 다양한 운영 환경에 녹여 넣는 작업의 복잡도가 **크게 감소**한다.

--- 

### MCP의 한계 ①: “좋은 에이전틱 인터페이스”는 API 설계에 달려 있다

MCP는 **“에이전틱 인터페이스(agentic interface)”를 위한 계약(contract)** 이다.  
그리고 MCP의 실효성은 **그 아래에 있는 API가 어떻게 설계되었는지**에 따라 크게 달라진다.

**에이전트가 잘 일하게 만들려면**, 기저 API가 다음과 같은 **결정론적(deterministic) 기능**을 제공해야 한다.

> 기존 레거시 API를 거의 손보지 않고  
> 그대로 MCP로 감싸기만 하는 경우

예시:

- 티켓 시스템 API가 “티켓 전체 내용을 하나씩 가져오기”만 지원한다고 하자.
- 그런 API를 MCP로 그냥 감싸놓고  
  에이전트에게 “우선순위 높은 티켓들을 요약해줘”라고 시키면,
  - 티켓이 많아질수록 하나씩 불러와야 하므로 **느리고 비효율적**이고
  - 누락이나 오류 가능성도 커진다.

**결정론적(deterministic) 기능**

- **필터링(filtering)**: 예) 우선순위가 높은 티켓만 한 번에 가져오기  
- **정렬(sorting)**: 예) 긴급도 순으로 정렬해서 가져오기  

이 점은 곧 다음을 의미한다.

> 에이전트가 기존의 결정론적 워크플로우를 “마법처럼 대체”하는 것이 아니라,  
> 오히려 **더 강력한 결정론적 지원이 있어야**  
> 비결정론적(non-deterministic) 에이전트가 제대로 일할 수 있다는 것.

### MCP의 한계 ②: 데이터 형식이 에이전트 친화적(agent-friendly)이어야 한다

MCP가 **단순히 API를 감싸는 역할**만 한다면,
그 API의 입력·출력 형식이 **에이전트가 이해하기 쉬운 형태임이 보장되지 않는다**

> “연결됐다고 해서 다 유용한 건 아니다.”

예시:

- 어떤 문서 저장소(document store)에 대해 MCP 서버를 만들었는데,  
  이 API가 문서를 **PDF 파일로만** 돌려준다고 하자.
- 이 때, 이를 소비하는 에이전트가 **PDF 내용을 파싱할 수 없다면**,  
  이 MCP 서버는 사실상 거의 쓸모가 없다.

더 나은 접근 방식은:

- 먼저 문서를 **텍스트 기반 형식(예: Markdown)** 으로 돌려주는 API를 설계하고  
- 그 API를 MCP 서버로 감싸는 것이다.

이렇게 해야:

- 에이전트가 실제로 내용을 **읽고, 이해하고, 가공**할 수 있고
- MCP 연결이 **“형식적으로만 연결된 상태”를 넘어, 실제로 유용한 통합**이 된다.

## MCP vs. Tool Function Calling(MCP와 툴 함수 호출 비교)

Model Context Protocol(MCP)와 **툴 함수 호출(tool function calling)** 은  
모두 LLM이 외부 기능(툴 포함)에 접근하고 액션을 실행하게 해주는 메커니즘이지만,  
접근 방식과 추상화 수준에서 차이가 있다.  
둘 다 “텍스트 생성 그 이상”을 가능하게 하지만, 역할과 범위가 다르다.

---

툴 함수 호출은 **LLM이 특정, 사전에 정의된 함수(=툴)를 직접 호출하는 방식**으로 이해할 수 있다.  
구조적으로는 **LLM ↔ 애플리케이션의 툴 처리 로직** 사이의 **일대일(one-to-one) 상호작용**에 가깝다.

- LLM은 사용자의 의도를 해석해  
    “외부 액션이 필요하다”고 판단하면,  
    그에 맞는 **함수 호출 요청을 포맷**해서 보낸다.
- 애플리케이션 코드는 이 요청을 받아 실제 툴/함수를 실행하고,  
    그 **결과를 다시 LLM에게 반환**한다.

이 프로세스는 **각 LLM 제공자마다 방식이 다르고, 벤더별·서비스별로 폐쇄적(proprietary)** 인 경우가 많다.

---

MCP는 LLM이 **외부 기능을 “발견하고(discover), 소통하며(communicate), 실제로 활용(utilize)”** 할 수 있게 해 주는 **표준화된 인터페이스**이다.  
즉, MCP를 따르면 어떤 툴·시스템이든 **같은 규약으로 노출**되고, MCP를 이해하는 어떤 LLM이든 **같은 방식으로 접근**할 수 있다.

이 프로토콜은 **오픈(open) 표준**으로 설계되어 있어,
- 서로 다른 LLM과 툴·시스템 간에 **상호운용성(interoperability)** 을 높이고  
- 여러 서비스를 조합해서 사용할 수 있는 **조합 가능성(composability)** 을 제공하며  
- 한 번 만든 MCP 서버를 여러 곳에서 재사용할 수 있는 **재사용성(reusability)** 을 촉진한다.

MCP는 또한 **연합(federated) 모델**을 채택한다.  
이는 각 서비스가 **자기 영역에서 독립적으로 계속 운영**되면서도,
- 그 위에 **MCP 호환 인터페이스만 하나 감싸면(wrapping)**  
- 기존의 서로 다른 시스템·레거시 서비스들을 **하나의 현대적인 에이전트 생태계 안으로 편입**시킬 수 있다는 뜻이다.

이렇게 MCP로 감싼 서비스들은:
- 내부 구현은 그대로 둔 채  
- LLM이 오케스트레이션하여 **새로운 애플리케이션과 워크플로우의 구성 요소**로 활용할 수 있다.  

결과적으로,
- **기존 핵심 시스템을 비싸게 갈아엎지 않고도**,  
- 다양한 자산과 레거시 시스템의 가치를 살리면서  
- 더 민첩하고(agility) 재사용 가능한 구조를 만들 수 있게 해 주는 전략이라고 볼 수 있다.

| 구분          | 툴 함수 호출 (Tool Function Calling)                                                | MCP (Model Context Protocol)                                                                                 |
|---------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| 표준화(Standardization) | 벤더별 독점·전용 방식으로, 형식과 구현이 LLM 제공자마다 다르다.                         | 오픈 표준 프로토콜로, 서로 다른 LLM과 툴 사이의 상호운용성을 촉진한다.                                      |
| 범위(Scope)   | LLM이 특정, **사전에 정의된 함수 하나**의 실행을 직접 요청하는 메커니즘이다.                   | LLM과 외부 툴·시스템이 **서로를 발견하고 소통하는 전체 방식**을 정의하는 더 넓은 프레임워크이다.           |
| 아키텍처(Architecture) | LLM과 애플리케이션의 툴 처리 로직 사이의 **1:1 상호작용 구조**이다.                     | LLM 기반 애플리케이션(클라이언트)이 여러 MCP 서버(툴)에 연결해 사용하는 **클라이언트–서버 아키텍처**이다. |
| 발견 방식(Discovery)   | 특정 대화 맥락에서 **어떤 툴이 사용 가능한지 미리 명시적으로 알려줘야** 한다.          | 사용 가능한 툴을 **동적으로 발견**할 수 있다. MCP 클라이언트가 서버에 어떤 기능을 제공하는지 질의할 수 있다. |
| 재사용성(Reusability)  | 툴 연동이 특정 애플리케이션과 특정 LLM에 **강하게 결합**되는 경우가 많다.              | 어떤 호환 애플리케이션에서도 접근 가능한 **재사용 가능한 독립형 MCP 서버** 개발을 장려한다.               |

툴 함수 호출(tool function calling)은,  
특정 작업을 위해 **맞춤 제작된 도구 세트**를 AI에게 쥐여주는 것과 비슷하다.  
예를 들면, 어떤 종류의 **렌치와 드라이버를 딱 정해진 용도로만 쓰는 작업장**을 떠올릴 수 있다.  

반면 **MCP(Model Context Protocol)** 는  
**범용·표준화된 전기 콘센트 시스템을 구축하는 것**에 가깝다.  
표준을 따르는 어떤 제조사의 도구든 **꽂기만 하면 바로 쓸 수 있게 해 주는 인프라**를 제공한다.  
이렇게 함으로써, 시간이 지날수록 다양한 도구가 추가되는  
**동적이고 계속 확장되는 작업장(workshop)** 을 만들 수 있다.

이제 MCP와 툴 함수 호출의 핵심적인 차이를 정리해 보면 다음과 같다.

- **함수 호출(Function Calling)** 은  
  - **소수의, 명확히 정의된 특정 함수들**에 직접 접근하는 메커니즘이다.
  - 비교적 **단순한 애플리케이션**이나,  
    사용해야 할 도구(기능) 세트가 **고정된 경우**에 적합하다.

- **MCP(Model Context Protocol)** 는  
  - LLM이 **다양한 외부 리소스, 툴, 시스템을 “발견하고 사용”** 할 수 있게 해 주는  
    **표준화된 통신 프레임워크**이다.
  - 여러 시스템이 서로 연결되고,  
    상황에 따라 다양한 서비스와 상호작용해야 하는 **복잡하고 유연한 AI 환경**에서는  
    이런 **범용 표준이 사실상 필수적인 기반**이 된다.

요약하자면,  
**함수 호출은 소수의 특정 기능에 대한 직통 통로**를 제공하고,  
**MCP는 LLM이 광범위한 외부 자원과 능력을 찾아내고 활용할 수 있게 하는 표준화된 연결 인프라**를 제공한다.  
단순한 서비스에는 특정 툴 몇 개면 충분하지만,  
**복잡하고 상호 연결된 에이전트 시스템**을 만들고 싶다면 MCP 같은 보편적 표준이 중요하다.

## **Additional Considerations for MCP(MCP 추가 고려 사항)**

- **Tool vs. Resource vs. Prompt:**

	세 구성요소의 역할을 명확히 이해하는 것이 중요하다.

	- **Resource(리소스):**  
	  정적 데이터를 의미한다.  
	  예: PDF 파일, 데이터베이스 레코드
	- **Tool(툴):**  
	  행동을 수행하는 실행 가능한 함수이다.  
	  예: 이메일 전송, API 쿼리
	- **Prompt(프롬프트):**  
	  LLM이 **리소스나 툴과 어떻게 상호작용해야 하는지**를 안내하는 템플릿이다.  
	  상호작용이 **구조화되고 일관되며 효과적인 방식**으로 이루어지도록 돕는다.

- **Discoverability(발견 가능성):**

	MCP의 큰 장점 중 하나는, **MCP 클라이언트가 서버에 동적으로 질의하여**  
	어떤 툴과 리소스를 제공하는지 **실시간으로 알아낼 수 있다**는 점이다.

	- 이와 같은 **“Just-in-time 발견” 메커니즘** 덕분에,
	- 에이전트는 새 기능이 추가되더라도 재배포 없이  
	  **새로운 능력을 스스로 발견하고 활용**할 수 있다.

- **Security(보안):**

	어떤 프로토콜이든 **툴과 데이터를 외부에 노출**하는 순간,  
	보안은 반드시 따라와야 하는 이슈이다.

	- MCP 구현 시에는 **인증(Authentication)** 과  
	  **인가(Authorization)** 를 통해,
		- 어떤 클라이언트가
		- 어떤 서버에
		- 어떤 액션까지 수행할 수 있는지  
		  명확히 통제해야 한다.

- **Implementation(구현 난이도):**

	MCP는 오픈 스탠다드이지만,  
	**구현이 복잡**할 수 있다.

	- 다만, 최근에는 구현 부담을 줄여주는 도구들이 등장하고 있다.  
	  예를 들어:
		- Anthropic
		- FastMCP  
	  같은 일부 모델/서비스 제공자는 **SDK나 라이브러리**를 제공하여,
		- 보일러플레이트 코드를 많이 숨기고
		- 개발자가 보다 쉽게 MCP 클라이언트·서버를 만들고 연결할 수 있도록 돕는다.

- **Error Handling(에러 처리):**

	**포괄적인 에러 처리 전략**은 MCP 설계에서 필수 요소이다.

	- 프로토콜 차원에서 다음과 같은 오류 상황을  
	  **어떻게 표현하고 LLM에게 전달할지** 정의해야 한다.
		- 툴 실행 실패
		- 서버 사용 불가(다운, 타임아웃 등)
		- 잘못된 요청(파라미터 오류, 권한 문제 등)
	- LLM이 **에러의 의미를 이해하고**,  
	  필요하다면 **대체 경로(다른 툴 사용, 재시도 등)** 를 시도할 수 있도록  
	  구조화된 에러 정보가 전달되어야 한다.

- **Local vs. Remote Server(로컬 vs. 원격 서버):**

	MCP 서버는 에이전트와 **같은 머신에 로컬로 배치**할 수도 있고,  
	**원격 서버로 분리**해서 운영할 수도 있다.

	- **로컬 서버(local):**
		- 민감한 데이터에 대해 **보안과 성능 측면에서 유리**할 수 있다.
		- 에이전트와 MCP 서버 간 통신 지연이 적다.
	- **원격 서버(remote):**
		- 여러 애플리케이션이 **공유 툴 세트**를 사용할 수 있는 구조를 만들 수 있다.
		- 공통 MCP 서버를 중심으로 **확장성과 운영 편의성**을 확보할 수 있다.

- **On-demand vs. Batch(온디맨드 vs. 배치 처리):**

	MCP는 **상호작용형(on-demand) 세션**과  
	**대규모 배치(batch) 처리** 모두를 지원할 수 있다.

	- 애플리케이션 특성에 따라  
	  MCP를 어떻게 호출할지(실시간/주기 실행/배치 작업 등)를 설계해야 한다.

- **Transportation Mechanism(전송 메커니즘):**

	MCP는 **통신에 사용할 전송 계층(transport layer)** 도 함께 정의한다.

	- **로컬 상호작용(Local):**
		- 같은 머신 내 프로세스 간 통신에는  
		  **STDIO(표준 입력/출력) 위의 JSON-RPC** 를 사용하여  
		  가볍고 효율적인 IPC(Inter-Process Communication)를 구현한다.
	- **원격 상호작용(Remote):**
		- 웹 환경에 친화적인 프로토콜을 사용한다.
			- **Streamable HTTP**
			- **Server-Sent Events(SSE)**  
		- 이를 통해 **지속적이고 효율적인 클라이언트–서버 통신**을 지원하며,  
		  스트리밍 응답 등에도 잘 대응할 수 있다.

The Model Context Protocol uses a **client–server 모델**을 통해 정보 흐름을 표준화한다.  
MCP에서 고급 에이전틱(agentic) 행동이 잘 작동하려면,  
아래 구성 요소들이 **어떻게 상호작용하는지 이해하는 것**이 핵심이다.

- **1. Large Language Model (LLM)**

	에이전트의 **핵심 지능**에 해당한다.  
	사용자의 요청을 이해하고, 수행할 계획을 세우며,  
	언제 외부 정보에 접근해야 하는지, 언제 어떤 액션을 실행해야 하는지를 결정한다.

- **2. MCP Client:**

	LLM을 감싸고 있는 **애플리케이션 또는 래퍼(wrapper)** 로,  
	LLM과 MCP 서버 사이에서 **중개자 역할**을 한다.

	- LLM이 표현한 “의도(intention)”를 MCP 표준에 맞는 **정형화된 요청(formal request)** 으로 변환한다.
	- 어떤 MCP 서버들이 존재하는지 **발견(discover)** 하고,
	- 해당 서버들과 **연결(connect)** 하며,
	- 요청·응답을 주고받는 **통신(communicate)** 을 담당한다.

- **3. MCP Server:**

	외부 세계로 향하는 **게이트웨이(gateway)** 이다.

	- MCP 클라이언트가 사용할 수 있는:
		- **Tools(툴)**  
		- **Resources(리소스)**  
		- **Prompts(프롬프트)**  
		  들을 노출한다.
	- 일반적으로 하나의 MCP 서버는 **특정 도메인**을 담당한다. 예를 들면:
		- 회사 내부 데이터베이스 연결
		- 이메일 서비스 연동
		- 외부 공개 API(예: 날씨 API) 래핑 등

- **4. Optional Third-Party (3P) Service:**

	실제 외부 **툴·애플리케이션·데이터 소스** 자체를 의미하는 선택적 구성 요소이다.  
	MCP 서버가 이 3rd-party 서비스를 **관리·위임 받아 노출**하는 구조다.

	- 예:  
		- 사내 전용 데이터베이스 쿼리
		- SaaS 플랫폼과 상호작용
		- 공공 날씨 API 호출 등  
	- 사용자가 요청한 액션을 **실제로 수행하는 최종 엔드포인트(endpoint)** 이며,  
	  MCP 서버는 그 앞단에서 이를 LLM 친화적인 형태로 감싸고 중계하는 역할을 한다.

MCP에서의 상호작용 흐름은 대략 다음 단계로 이루어진다.

- **1. Discovery(발견 단계):**

	MCP 클라이언트는 LLM을 대신해 **MCP 서버에 질의**하여  
	“어떤 기능(capabilities)을 제공하는지”를 묻는다.  

	- MCP 서버는 다음과 같은 내용을 담은 **매니페스트(manifest)** 를 응답으로 돌려준다.
		- 사용 가능한 **툴(tools)** 예: `send_email`
		- 접근 가능한 **리소스(resources)** 예: `customer_database`
		- 제공하는 **프롬프트(prompts)** 목록 등  

	이 단계에서 LLM은 “이 서버가 무엇을 할 수 있는지”에 대한 **기능 지도(capability map)** 를 얻게 된다.

- **2. Request Formulation(요청 구성):**

	이제 LLM은 **발견된 툴 중 하나를 실제로 사용해야 한다고 판단**한다.  

	예를 들어:
	- 사용자의 요청을 분석한 결과, 이메일을 보내야 한다고 결정했다고 하자.
	- LLM은 `send_email` 툴을 사용하기로 결정하고,
	- 여기에 필요한 **파라미터들**을 채운다.
		- 수신자(recipient)
		- 제목(subject)
		- 본문(body) 등  

	이렇게 해서 LLM은 **“어떤 툴을 어떤 인자로 호출할지”가 명시된 요청 객체**를 구성한다.

- **3. Client Communication(클라이언트 통신):**

	MCP 클라이언트는 LLM이 구성한 요청을 받아,  
	이를 MCP 표준에 맞는 **정규화된 호출(standardized call)** 로 변환한다.  

    해당 요청을 **적절한 MCP 서버**에 전송한다.

- **4. Server Execution(서버 실행):**

	MCP 서버는 클라이언트로부터 요청을 수신한 뒤:

	1. **클라이언트 인증(Authentication)** 을 수행하고  
	2. 요청이 유효한지(권한, 파라미터 형식, 리소스 접근 등) **검증(Validation)** 한 다음  
	3. 실제 **지정된 액션을 실행**한다.

	예를 들어 이메일 전송의 경우:
	- 내부적으로 연결된 이메일 API의 `send()` 함수를 호출하여  
	  실제 메일 발송을 수행한다.

- **5. Response and Context Update(응답 및 컨텍스트 업데이트):**

	액션 실행이 끝나면, MCP 서버는 결과를 **표준화된 응답 형식**으로 MCP 클라이언트에 반환한다.

	- 응답에는 보통 다음 정보가 포함된다.
		- 작업 성공 여부(success / failure)
		- 관련 출력 데이터 (예: 발송 완료된 이메일의 확인 ID, 처리 결과 값 등)

	MCP 클라이언트는 이 응답을 다시 LLM에게 전달하고,  
	LLM은 이를 자신의 **컨텍스트(context)** 에 반영한다.

	이렇게 업데이트된 컨텍스트를 기반으로 LLM은:
	- 다음 단계 계획을 세우고
	- 후속 액션을 결정하거나
	- 사용자에게 최종 응답을 생성하는 등  
	**전체 작업 플로우를 이어서 수행**하게 된다.

## Practical Applications & Use Cases  
MCP는 AI/LLM의 능력을 크게 확장하여 **더 유연하고 강력한 에이전트**를 만들 수 있게 한다.  

- **Database Integration(데이터베이스 통합):**

	MCP를 사용하면 LLM과 에이전트가 **구조화된 데이터베이스와 자연스럽게 상호작용**할 수 있다.  
	예를 들어, MCP Database Toolbox를 통해 에이전트가 Google BigQuery 같은 데이터셋에 쿼리를 보내  
	실시간 정보를 조회하고, 보고서를 생성하거나, 레코드를 갱신하는 작업을  
	**자연어 명령만으로 수행**할 수 있다.

- **Generative Media Orchestration(생성 미디어 오케스트레이션):**

	MCP는 에이전트가 **고급 생성형 미디어 서비스**와 연동되도록 돕는다.  
	MCP 기반 Genmedia 도구들을 사용하면 에이전트는 다음과 같은 워크플로를 구성할 수 있다.
	- 이미지 생성: Google **Imagen**
	- 동영상 생성: Google **Veo**
	- 현실감 있는 음성: Google **Chirp 3 HD**
	- 음악 작곡: Google **Lyria**  
	이를 통해 AI 애플리케이션 안에서 **동적으로 이미지, 영상, 음성, 음악을 생성·조합**하는 시나리오를 구현할 수 있다.

- **External API Interaction(외부 API 연동):**

	MCP는 LLM이 **어떤 외부 API든 표준화된 방식으로 호출하고 응답을 받을 수 있는 통로**를 제공한다.  
	예를 들어, 에이전트는 MCP를 통해:
	- 실시간 날씨 정보 조회
	- 주가/시세 데이터 가져오기
	- 이메일 전송
	- CRM(고객 관리 시스템)과 상호작용  
	등을 수행할 수 있으며, 이를 통해 **언어 모델의 한계를 넘어 실제 시스템과 연결된 에이전트**로 확장된다.

- **Reasoning-Based Information Extraction(추론 기반 정보 추출):**

	MCP는 LLM의 강력한 **추론 능력**을 활용해,  
	단순 검색을 넘어 **쿼리 의존적인 정보 추출**을 가능하게 한다.

	- 기존 검색 도구처럼 문서 전체를 통으로 반환하는 대신,  
	- 에이전트가 텍스트를 직접 분석하여  
	  사용자의 복잡한 질문에 **정확히 대응하는 문장, 조항, 수치, 그림만 발췌**할 수 있다.  

- **Custom Tool Development(커스텀 툴 개발):**

	개발자는 자신만의 **커스텀 툴**을 만들고 MCP 서버(예: FastMCP 기반)를 통해 노출할 수 있다.

	- 사내 전용 함수나
	- 폐쇄적인 내부 시스템,
	- 독자적인 비즈니스 로직 등을  
	  LLM이 사용할 수 있는 **표준화된 MCP 툴**로 감쌀 수 있다.
	- 이 과정에서 LLM을 직접 수정하거나, 모델을 다시 훈련할 필요 없이  
	  **외부 레이어에서 기능을 확장**할 수 있다

- **Standardized LLM-to-Application Communication (LLM–애플리케이션 간 표준화된 통신):**

	MCP는 LLM과 애플리케이션 사이에 **일관된 통신 레이어**를 제공한다.

	- 각 LLM 제공자별, 서비스별로 다른 인터페이스를 직접 붙이는 대신,
	- MCP라는 공통 표준 위에 얹어두면  
	  **통합 비용이 줄어들고**,  
	  서로 다른 LLM·호스트 애플리케이션 간 **상호운용성**이 높아진다.  
	그 결과, **복잡한 에이전트 시스템**을 더 쉽게 설계·구현할 수 있다.

- **Complex Workflow Orchestration(복잡한 워크플로 오케스트레이션):**

	여러 MCP 툴과 데이터 소스를 조합하면,  
	에이전트가 **멀티 스텝과 복잡한 워크플로를 자동으로 오케스트레이션**할 수 있다.

	예를 들어, 하나의 에이전트가 다음을 연속적으로 수행할 수 있다.
	1. 데이터베이스에서 고객 데이터를 조회하고  
	2. 개인화된 마케팅 이미지를 생성하고  
	3. 그 고객에게 맞춤형 이메일을 작성한 뒤  
	4. 실제 이메일을 발송하는 것까지  

	이 모든 단계가 **각기 다른 MCP 서비스들과의 상호작용**으로 구성될 수 있다.

- **IoT Device Control(IoT 디바이스 제어):**

	MCP는 LLM이 **사물인터넷(IoT) 디바이스와 상호작용**하는 것도 가능하게 한다.

	- 스마트 홈 가전 제어
	- 산업용 센서 모니터링 및 제어
	- 로봇 장비 제어 등  
	에이전트는 자연어 명령을 MCP를 통해 **실제 물리 시스템에 대한 제어 신호**로 변환할 수 있어,  
	자연어 기반 자동화 시나리오를 구축할 수 있다.

- **Financial Services Automation(금융 서비스 자동화):**

	금융 도메인에서 MCP는 LLM이 다양한:
	- 금융 데이터 소스,
	- 트레이딩 플랫폼,
	- 컴플라이언스(규제 준수) 시스템  
	과 상호작용하도록 돕는다.

	에이전트는 예를 들어:
	- 시장 데이터를 분석하고
	- 거래를 실행하며
	- 개인화된 투자·재무 조언을 생성하고
	- 규제 보고를 자동화하는 작업까지 수행할 수 있다.  
	이 모든 과정에서 MCP는 **보안이 보장된 표준화된 커뮤니케이션 경로**를 제공한다.

---

### 요약

Model Context Protocol(MCP)는 에이전트가:

- 데이터베이스, 외부 API, 웹 리소스 등에서 **실시간 정보를 조회**하고,
- 이메일 전송, 레코드 업데이트, 디바이스 제어 등 **실제 액션을 수행**하며,
- 여러 소스의 데이터를 통합·처리해 **복잡한 작업을 자동화**하도록 만들어 준다.

또한, 생성형 미디어 도구와의 연동까지 지원하여  
**텍스트를 넘어 이미지·영상·음성·음악까지 다루는 리치한 AI 애플리케이션**을 구축하는 기반을 제공한다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

In [6]:
client = MultiServerMCPClient(
    {
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",  # npx 설치 자동 승인
                "@modelcontextprotocol/server-filesystem",
                ".",  # 반드시 절대 경로여야 함
            ],
            "transport": "stdio",
        }
    })
tools = await client.get_tools()

In [7]:
tools

[StructuredTool(name='read_file', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', args_schema={'$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13c197880>),
 StructuredTool(name='read_text_file', description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only works within allowed directories.", args_schema={'$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langc

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.0,
)

agent = create_agent(
    llm,
    tools
)

file_response = await agent.ainvoke(
    {
        "messages": [
            {"role": "user", "content": "현재 디렉토리에 있는 파일들을 보여줘."}
        ]
    }
)

I0000 00:00:1763621039.611874 2864023 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [12]:
print(file_response)

{'messages': [HumanMessage(content='현재 디렉토리에 있는 파일들을 보여줘.', additional_kwargs={}, response_metadata={}, id='0f91a67c-86bc-4b2b-a575-9294db06893d'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'list_directory', 'arguments': '{"path": "."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dce3ef86-cca9-4032-ae34-fa28e2e1932e-0', tool_calls=[{'name': 'list_directory', 'args': {'path': '.'}, 'id': 'a762b769-5e24-42f5-989e-299213772aed', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1037, 'output_tokens': 70, 'total_tokens': 1107, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 56}}), ToolMessage(content='[FILE] .env\n[DIR] .git\n[FILE] .gitignore\n[FILE] .python-version\n[DIR] .venv\n[FILE] Qwen_Qwen3-4B-Q4_K_M.gguf\n[FILE] README.md\n[DIR] 

In [13]:
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition


def call_model(state: MessagesState):
    response = llm.bind_tools(tools).invoke(state["messages"])
    return {"messages": state["messages"] + [response]}


build_graph = StateGraph(MessagesState)
build_graph.add_node("model", call_model)
build_graph.add_node("tools", ToolNode(tools))

build_graph.add_edge(START, "model")
build_graph.add_conditional_edges(
    "model",
    tools_condition
)
build_graph.add_edge("tools", "model")

graph = build_graph.compile()

result = await graph.ainvoke(
    {
        "messages": [
            {"role": "user", "content": "현재 디렉토리에 있는 파일들을 보여줘."}
        ]
    }
)

result["messages"]

I0000 00:00:1763621062.965307 2864023 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


[HumanMessage(content='현재 디렉토리에 있는 파일들을 보여줘.', additional_kwargs={}, response_metadata={}, id='cdbab58c-dced-4479-a7a3-db3743046e62'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'list_directory', 'arguments': '{"path": "."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--8828244b-14f5-48a4-9530-cef0b0a2ecce-0', tool_calls=[{'name': 'list_directory', 'args': {'path': '.'}, 'id': 'cda57386-b80c-4e5c-82e6-1c2016efd429', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1037, 'output_tokens': 71, 'total_tokens': 1108, 'input_token_details': {'cache_read': 730}, 'output_token_details': {'reasoning': 57}}),
 ToolMessage(content='[FILE] .env\n[DIR] .git\n[FILE] .gitignore\n[FILE] .python-version\n[DIR] .venv\n[FILE] Qwen_Qwen3-4B-Q4_K_M.gguf\n[FILE] README.md\n[DIR] __pycache

## FastMCP

FastMCP는 모델 컨텍스트 프로토콜(MCP) 서버를 빠르게 만들 수 있도록 설계된 파이썬 프레임워크

```sh
uv run fastmcp_greet.py
```

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL_NAME = "gemini-2.5-flash"

In [41]:
import asyncio
from typing import List, Optional
from typing_extensions import TypedDict, Annotated
import operator

from langchain_core.messages import (
    HumanMessage,
    AIMessage,
    ToolMessage,
    BaseMessage,
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.types import StreamWriter

In [42]:
class AgentState(TypedDict):
    # 전체 대화/툴 호출 히스토리 → memory
    messages: Annotated[List[BaseMessage], add_messages]
    # planner가 만든 단계별 계획
    plan: Optional[List[str]]
    # 현재 실행 중인 단계 인덱스
    step_index: int
    # reflection 결과 요약
    reflection: Optional[str]

In [43]:
async def load_mcp_tools():
    mcp_url = os.getenv("MCP_SERVER_URL", "http://localhost:7777/mcp")

    client = MultiServerMCPClient(
        {
            "fastmcp": {
                "transport": "streamable_http",
                "url": mcp_url,
            }
        }
    )
    tools = await client.get_tools()

    tool_names = [tool.name for tool in tools]
    print(f"MCP 서버에서 로드된 도구들: {tool_names}")

    greet_tools = [tool for tool in tools if tool.name == "greet"]
    if not greet_tools:
        raise RuntimeError("MCP 서버에서 `greet` 도구를 찾을 수 없습니다.")
    return greet_tools[0]

In [44]:
def make_planner_llm() -> ChatGoogleGenerativeAI:
    return ChatGoogleGenerativeAI(
        model=GEMINI_MODEL_NAME,
        api_key=GEMINI_API_KEY,
        temperature=0.2,
    )

def make_actor_llm() -> ChatGoogleGenerativeAI:
    return ChatGoogleGenerativeAI(
        model=GEMINI_MODEL_NAME,
        api_key=GEMINI_API_KEY,
        temperature=0.7,
    )

def make_reflector_llm() -> ChatGoogleGenerativeAI:
    return ChatGoogleGenerativeAI(
        model=GEMINI_MODEL_NAME,
        api_key=GEMINI_API_KEY,
        temperature=0.0,
    )

In [48]:
def planner_node(state: AgentState) -> AgentState:
    """user 요청 + memory 기반으로 단계별 plan을 만든다"""
    llm = make_planner_llm()

    history_text = "\n".join(
        f"{msg.type}: {msg.content}" for msg in state["messages"]
    )

    prompt = f"""
    당신은 MCP 서버를 통해 외부 도구를 사용하는 에이전트의 '플래너'입니다.

    - MCP 서버에는 `greet(name: str) -> str` 이라는 도구가 있습니다.
    - 이 도구는 이름을 받아 자연스러운 인사 문장을 생성합니다.
    - 도구를 직접 호출하지는 않고, 실행 노드가 이 도구를 호출한다고 가정하세요.

    대화/시스템 기록:
    {history_text}

    1. 사용자의 목표를 한 줄로 요약하고,
    2. 그 목표를 달성하기 위해 어떤 순서로 MCP greet 도구를 활용할지
    1~3단계의 bullet list로 계획을 세우세요.
    3. 각 단계는 한국어로, 간단명료하게 작성하세요.
    """

    resp = llm.invoke([HumanMessage(content=prompt)])
    print(f"원본 응답:\n{resp.content}")

    lines = str(resp.content).splitlines()
    steps = [line.lstrip("-•* ").strip() for line in lines if line.strip().startswith(("-", "•", "*"))]
    print(f"플래너가 생성한 단계별 계획: {steps}")

    return {
        **state,
        "plan": steps,
        "step_index": 0,
        "reflection": state.get("reflection"),
        "messages": [
            AIMessage(
                content="계획:\n" + "\n".join(f"- {s}" for s in steps)
            )
        ]
    }

async def actor_node(state: AgentState, *, mcp_greet_tool) -> AgentState:
    """현재 단계(plan[step_index])를 실행. 여기서는 greet MCP 도구 호출."""
    plan = state.get("plan", [])
    idx = state.get("step_index", 0)

    if idx >= len(plan):
        return {
            "plan": plan,
            "step_index": idx,
            "reflection": state.get("reflection"),
        }

    current_step = plan[idx]

    llm = make_actor_llm()
    history_text = "\n".join(
        f"{msg.type}: {getattr(m, 'content', '')}" for m in state["messages"]
    )

    prompt = f"""
    다음은 에이전트와 사용자의 대화 기록이다:

    {history_text}

    위 대화를 보고, 인사해야 할 사람의 이름만 정확히 한 단어 또는 두 단어로 출력해라.
    이름이 없다면 'UNKNOWN'만 출력해라.
    """

    resp = llm.invoke([HumanMessage(content=prompt)])
    name = str(resp.content).strip()

    if name.upper() == "UNKNOWN":
        greet_result = "이름을 알 수 없어 인사를 할 수 없습니다."
    else:
        greet_result = await mcp_greet_tool.ainvoke({"name": name})

    tool_msg = ToolMessage(
        content=str(greet_result),
        tool_call_id="mcp-greet-call",
    )

    return {
        "plan": plan,
        "step_index": idx + 1,
        "reflection": state.get("reflection"),
        "messages": [
            AIMessage(content=f"현재 단계 실행: {current_step}\n대상 이름: {name}"),
            tool_msg,
        ]
    }

def reflector_node(state: AgentState) -> AgentState:
    """툴 결과를 보고 인사가 자연스러운지, 개선할 점이 있는지 reflection."""
    llm = make_reflector_llm()

    tool_msgs = [m for m in state["messages"] if isinstance(m, ToolMessage)]
    if not tool_msgs:
        return {
            "plan": state.get("plan"),
            "step_index": state.get("step_index", 0),
            "reflection": state.get("reflection"),
        }

    last_tool = tool_msgs[-1]

    prompt = f"""
    다음은 에이전트가 MCP greet 도구를 사용해 생성한 인사 메시지이다:

    \"\"\"{last_tool.content}\"\"\"

    1. 자연스러운 한국어 인사인가?
    2. 더 나은 버전이 있다면 한 줄로 다시 적어라.
    3. 충분히 괜찮다면 'OK'만 출력해라.
    """

    resp = llm.invoke([HumanMessage(content=prompt)])
    reflection = str(resp.content).strip()

    new_messages: List[BaseMessage] = []
    if reflection != "OK":
        new_messages = new_messages.append(
            AIMessage(content=f"[Reflection 개선안]\n{reflection}")
        )

    if new_messages:
        return {
            "plan": state.get("plan"),
            "step_index": state.get("step_index", 0),
            "reflection": reflection,
            "messages": new_messages,
        }
    else:
        return {
            "plan": state.get("plan"),
            "step_index": state.get("step_index", 0),
            "reflection": "OK",
        }

def should_reflect_or_end(state: AgentState) -> str:
    """reflection 이후 그래프를 종료할지 결정."""
    return END

In [49]:
async def build_graph():
    greet_tool = await load_mcp_tools()

    async def _actor_node_wrapped(state: AgentState) -> AgentState:
        return await actor_node(state, mcp_greet_tool=greet_tool)

    graph = StateGraph(AgentState)

    graph.add_node("planner", planner_node)
    graph.add_node("actor", _actor_node_wrapped)
    graph.add_node("reflector", reflector_node)

    graph.add_edge(START, "planner")
    graph.add_edge("planner", "actor")
    graph.add_edge("actor", "reflector")
    graph.add_conditional_edges(
        "reflector",
        should_reflect_or_end,
        {END: END}
    )

    return graph.compile()

In [50]:
graph = await build_graph()

init_state: AgentState = {
    "messages": [
        HumanMessage(content="안녕, 내 이름은 철수야. 반가워!")
    ],
    "plan": None,
    "step_index": 0,
    "reflection": None,
}

result = await graph.ainvoke(init_state)

for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

MCP 서버에서 로드된 도구들: ['greet']
원본 응답:
1.  **사용자의 목표:** 사용자에게 이름으로 인사하기.

2.  **MCP greet 도구 활용 계획:**
    *   사용자 발화에서 이름 '철수'를 추출합니다.
    *   추출한 이름('철수')을 인자로 사용하여 MCP 서버의 `greet` 도구를 호출합니다.
    *   도구의 반환 값을 사용자에게 응답으로 전달합니다.
플래너가 생성한 단계별 계획: ["사용자 발화에서 이름 '철수'를 추출합니다.", "추출한 이름('철수')을 인자로 사용하여 MCP 서버의 `greet` 도구를 호출합니다.", '도구의 반환 값을 사용자에게 응답으로 전달합니다.']
human: 안녕, 내 이름은 철수야. 반가워!

ai: 계획:
- 사용자 발화에서 이름 '철수'를 추출합니다.
- 추출한 이름('철수')을 인자로 사용하여 MCP 서버의 `greet` 도구를 호출합니다.
- 도구의 반환 값을 사용자에게 응답으로 전달합니다.

ai: 현재 단계 실행: 사용자 발화에서 이름 '철수'를 추출합니다.
대상 이름: 철수

tool: Hello, 철수! Welcome to FastMCP.

